In [4]:
import math

## Complexity

| Type | Average | Worst | Storage | Stable? |
| -- | -- | -- | -- | -- |
| Insertion | $\Theta(n^2)$ | $\Theta(n^2)$ | $\Theta(n)$ | Stable |
| Merge | $\Theta(n \log n)$ | $\Theta(n \log n)$ | $\Theta(n)$ | Stable |
| Heap | ~ | $O(n \log n)$ | $\Theta(n)$ | ! |
| Quick | $\Theta(n \log n)$ | $\Theta(n^2)$ | $\Theta(n)$ | ! |
| Counting | $\Theta(k + n)$ | $\Theta(k + n)$ | $\Theta(k)$ | Stable |
| Radix | $\Theta(d(k + n))$ | $\Theta(d(k + n))$ | | Stable (if CS) |
| Bucket | $\Theta(n)$ | $\Theta(n^2)$ | | Stable (if IS) |


## Insertion Sort

In [5]:
def insertion_sort(S: list):
    for j in range(1, len(S)):
        ## start with item index 1 and go through each index
        key = S[j]
        i = j-1
        while i >= 0 and S[i] > key:
            ## loop backwards through sorter region until end or until list item is greater than key
            ## shift items to make space for insert 
            S[i+1] = S[i]
            i = i - 1
        S[i+1] = key

## Merge Sort

In [6]:
def mergesort_merge(S, p, q, r):
    ## extract the two sorted sublists were trying to merge
    L = [S[p]] if p == q else S[p:q+1] 
    R = [S[q+1]] if q+1 == r else S[q+1:r+1]
    # trick to simplify the merge
    # if we append both with infinity, if one list ends 
    # the other will be appended without any logic change 
    L.append(float("inf"))
    R.append(float("inf"))
    i = 0 # L index
    j = 0 # R index
    # k is the index in the top level list we're replacing
    for k in range(p, r+1):
        if L[i] <= R[j]:
            S[k] = L[i]
            i += 1
        else:
            S[k] = R[j]
            j += 1

def mergesort(S, p = None, r = None):
    if p == None:
        p = 0
    if r == None:
        r = len(S)-1
    if p < r:
        # S is the list, p-r is the sub-array index to sort
        q = math.floor((p+r)/2) # midpoint
        # sort these two sub-arrays
        mergesort(S, p, q)
        mergesort(S, q + 1, r)
        mergesort_merge(S, p, q, r)
    else:
        # a single item is already sorted - do nothing
        pass

## Heapsort

In [7]:
def max_heapify(A, heap_size, i):
    """ recursive violation solver for a max heap A"""
    def left_child(i):
        return 2*i + 1

    def right_child(i):
        return 2*i + 2
    
    left = left_child(i)
    right = right_child(i)
    max_i = i

    if left < heap_size and A[left] > A[max_i]:
        max_i = left
    if right < heap_size and A[right] > A[max_i]:
        max_i = right
    if max_i != i:
        A[i], A[max_i] = A[max_i], A[i]
        max_heapify(A, heap_size, max_i)

def build_mh(A):
    size = len(A)
    for i in range(size//2 - 1, -1, -1):
        max_heapify(A, size, i)
    return A

def heapsort(A): # Floyd
    build_mh(A)
    heap_size = len(A)
    while heap_size > 1:
        A[heap_size - 1], A[0] = A[0], A[heap_size - 1]
        heap_size -= 1
        max_heapify(A, heap_size, 0)

## Quicksort

In [8]:
def partition(A, low, high):
    return lomuto(A, low, high)

def lomuto(A, low, high):
    pv = high
    pv_val = A[pv]
    x = low # pivot position

    for j in range(low, high):
        if A[j] < pv_val:
            A[x], A[j] = A[j], A[x]
            x = x +1
    A[x], A[pv] = A[pv], A[x]
    return x

def quicksort(A, low = None, high = None):
    if low is None:
        low = 0
    if high is None:
        high = len(A)-1

    if low < high:
        pivot = partition(A, low, high)
        quicksort(A, low, pivot-1)  #LSA
        quicksort(A, pivot+1, high) #RSA

## Counting Sort


In [9]:
def counting_sort(A, k = None):
    if k is None:
        k = max(A) + 1

    n = len(A)
    counts = [0 for x in range(k)]
    output = [0 for x in range(n)]

    for key in A:
        counts[key] += 1
    print(f"Counts: {counts}")
    for i in range(1, k):
        counts[i] += counts[i - 1]
    print(f"Running: {counts}")
    for key in reversed(A):
        output[counts[key]-1] = key
        counts[key] = counts[key] - 1
    return output

## LSD Radix Sort

In [10]:
def stable_sort(A, k, dp):

    def gd(num, n): # 0 -> LSD
        return num // 10**n % 10

    # use counting sort as a stable sort
    n = len(A)
    counts = [0 for x in range(k)]
    output = [0 for x in range(n)]

    for key in A:
        counts[gd(key, dp)] += 1

    for i in range(1, k):
        counts[i] += counts[i - 1]

    for key in reversed(A):
        output[counts[gd(key, dp)]-1] = key
        counts[gd(key, dp)] = counts[gd(key, dp)] - 1
    return output

def LSD_radix_sort(A, d, k):
    for digit_pos in range(d):
        A = stable_sort(A, k, digit_pos)
    return A

## Bucket Sort

In [11]:
def bucket_sort(A):
    num_buckets = len(A)
    buckets = [[] for _ in range(num_buckets)]

    for key in A: # Scatter
        buckets[int(num_buckets * key)].append(key)
    for bucket in buckets:
        insertion_sort(bucket)
    return [x for bucket in buckets for x in bucket] # gather